In [ ]:
import psycopg2
import pandas as pd
from datetime import timedelta

In [ ]:
# Configurações de conexão
host = 'postgresql-datadt.alwaysdata.net'
database = 'datadt_digital_corporativo'
user = 'datadt_data_analytics'
password = 'DataAnalytics$100'

In [ ]:
# Conectar ao PostgreSQL
try:
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )
    print('Conexão estabelecida com sucesso!')
    
    # Criar cursor
    cursor = conn.cursor()
    
except Exception as e:
    print(f'Erro ao conectar: {e}')

In [ ]:
# Consulta SQL
query = """
select nf.id, nf.data_venda::date as data, nf.valor , COALESCE(pf.cpf, pj.cnpj) AS cpf_cnpj 
from vendas.nota_fiscal nf 
left join geral.pessoa_fisica pf on pf.id = nf.id_cliente 
left join geral.pessoa_juridica pj on pj.id = nf.id_cliente
"""

# Executar consulta e criar DataFrame
df = pd.read_sql_query(query, conn)
print(f'DataFrame criado com {len(df)} registros')
df.head()

In [ ]:
# Data de referência para cálculo da Recência
data_referencia = df['data'].max()
print(f'Data de referência: {data_referencia}')

# Calcular métricas RFM por cliente
rfm = df.groupby('cpf_cnpj').agg({
    'data': lambda x: (data_referencia - x.max()).days,
    'id': 'count',
    'valor': 'sum'
}).reset_index()

rfm.columns = ['cpf_cnpj', 'recencia', 'frequencia', 'valor_monetario']
print(f'RFM calculado para {len(rfm)} clientes')
rfm.head()

In [ ]:
# Criar scores RFM (1-5)
rfm['r_score'] = pd.qcut(rfm['recencia'], 5, labels=[5,4,3,2,1])
rfm['f_score'] = pd.qcut(rfm['frequencia'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['m_score'] = pd.qcut(rfm['valor_monetario'], 5, labels=[1,2,3,4,5])

# Score RFM combinado
rfm['rfm_score'] = rfm['r_score'].astype(str) + rfm['f_score'].astype(str) + rfm['m_score'].astype(str)

rfm.head()

In [ ]:
# Segmentação simplificada
def segmentar_cliente(row):
    r, f, m = int(row['r_score']), int(row['f_score']), int(row['m_score'])
    
    if r >= 4 and f >= 4 and m >= 4:
        return 'Campeões'
    elif r >= 3 and f >= 3 and m >= 3:
        return 'Clientes Leais'
    elif r >= 4 and f <= 2:
        return 'Novos Clientes'
    elif r <= 2 and f >= 3 and m >= 3:
        return 'Em Risco'
    elif r <= 2 and f <= 2:
        return 'Perdido'
    else:
        return 'Clientes Potenciais'


In [ ]:
rfm['segmento'] = rfm.apply(segmentar_cliente, axis=1)

In [ ]:
rfm.head()

In [ ]:
# Análise dos segmentos
segmentos = rfm.groupby('segmento').agg({
    'cpf_cnpj': 'count',
    'recencia': 'mean',
    'frequencia': 'mean',
    'valor_monetario': 'mean'
}).round(2)

segmentos.columns = ['qtd_clientes', 'recencia_media', 'frequencia_media', 'valor_medio']
segmentos['percentual'] = (segmentos['qtd_clientes'] / len(rfm) * 100).round(1)

print('Análise por Segmento:')
segmentos.sort_values('qtd_clientes', ascending=False)

## Mais passo a passo

In [ ]:
df.info()

In [ ]:
data_mais_atual = df['data'].max() + timedelta(days=1)
data_mais_atual

In [ ]:
df_agregado = df.groupby('cpf_cnpj').agg({'data': lambda x: (data_mais_atual - x.max()).days,
                                                   'id': 'count',
                                                   'valor': 'sum'})

In [ ]:
df_agregado

In [ ]:
df_agregado.shape

In [ ]:
df['cpf_cnpj'].nunique()

In [ ]:
df_agregado.info()

In [ ]:
df_agregado.rename(columns={'data': 'recencia', 
                            'id': 'frequencia', 
                            'valor': 'monetario'}, inplace=True)

In [ ]:
df_agregado

In [ ]:
r_groups = pd.qcut(df_agregado['recencia'], q=5, labels=range(1,6))
r_groups

In [ ]:
f_groups = pd.qcut(df_agregado['frequencia'], q=5, labels=range(1,6))
f_groups

In [ ]:
m_groups = pd.qcut(df_agregado['monetario'], q=5, labels=range(1,6))
m_groups

In [ ]:
df_agregado['R'] = r_groups
df_agregado['F'] = f_groups
df_agregado['M'] = m_groups

In [ ]:
df_agregado.info()

In [ ]:
df_agregado['R' ] = df_agregado['R'].astype(int)
df_agregado['F'] = df_agregado['F'].astype(int)
df_agregado['M'] = df_agregado['M'].astype(int)

In [ ]:
df_agregado['R'].value_counts()

In [ ]:
df_agregado['F'].value_counts()

In [ ]:
df_agregado['M'].value_counts()

In [ ]:
df_agregado['RFM_Score'] = df_agregado['R'] + df_agregado['F'] + df_agregado['M']
df_agregado['RFM_Segment'] = df_agregado['R'].astype(str) + df_agregado['F'].astype(str) + df_agregado['M'].astype(str)

In [ ]:
df_agregado[['R', 'F', 'M', 'RFM_Score', 'RFM_Segment']].head()

In [ ]:
def classify_by_score(score):
    if score >= 11:
        return 'Alto Valor'
    elif score >= 7:
        return 'Médio Valor'
    else:
        return 'Baixo Valor'

def classify_by_segment(segment):
    r, f, m = int(segment[0]), int(segment[1]), int(segment[2])
    if r >= 4 and f >= 4 and m >= 4:
        return 'Campeões'
    elif r >= 3 and f >= 3 and m >= 3:
        return 'Clientes Fiéis'
    elif r >= 3 and f >= 1 and m >= 3:
        return 'Clientes Potenciais'
    elif r >= 3 and f >= 1 and m >= 1:
        return 'Novos Clientes'
    elif r >= 1 and f >= 1 and m >= 1:
        return 'Clientes em Risco'
    else:
        return 'Perdidos'

In [ ]:
df_agregado['Classe_Score'] = df_agregado['RFM_Score'].apply(classify_by_score)
df_agregado['Classe_Segmento'] = df_agregado['RFM_Segment'].apply(classify_by_segment)

In [ ]:
df_agregado

In [ ]:
print("Distribuição por Classe Score:")
print(df_agregado['Classe_Score'].value_counts())
print("\nDistribuição por Classe Segmento:")
print(df_agregado['Classe_Segmento'].value_counts())

In [ ]:
def acao_por_segmento(classe):
    if classe == 'Campeões':
        return 'Priorizar: Oferecer benefícios exclusivos e programas VIP para maximizar retenção e valor.'
    elif classe == 'Clientes Fiéis':
        return 'Manter: Enviar comunicações regulares e ofertas de upsell para fortalecer o relacionamento.'
    elif classe == 'Clientes Potenciais':
        return 'Converter: Incentivar compras mais frequentes com promoções personalizadas e cross-sell.'
    elif classe == 'Novos Clientes':
        return 'Integrar: Bem-vindo e orientações para construir lealdade desde o início.'
    elif classe == 'Clientes em Risco':
        return 'Reativar: Campanhas de win-back com descontos e lembretes para recuperar interesse.'
    else:
        return 'Avaliar: Considerar se vale a pena investir em reativação ou focar em segmentos mais promissores.'

In [ ]:
df_agregado['Acao'] = df_agregado['Classe_Segmento'].apply(acao_por_segmento)

In [ ]:
df_agregado.sample(10)

In [ ]:
df_agregado

In [ ]:
from enviar_email import EnviarEmail

# Configurações de email
remetente = ''
senha = ''
destinatario = ''
servidor_smtp = 'smtp.gmail.com'
porta_smtp = 587

# Criar instância da classe
email_sender = EnviarEmail(servidor_smtp, porta_smtp, remetente, senha)

# Enviar relatório RFM
email_sender.enviar(
    assunto='Relatório RFM - Análise de Clientes',
    df=df_agregado.head(15),
    destinatario=destinatario
)

print('Email enviado com sucesso!')
